17.1 Getting the Data

When trying to run the read_html code this error is displayed

"ImportError: lxml not found, please install"

Anacoda showed it already installed but you need to run this command at the cmd prompt to reinstall.

"conda install -c anaconda lxml"




In [1]:
import pandas as pd


url = 'https://en.wikipedia.org/wiki/Historical_rankings_of_presidents_of_the_United_States'

pres_dfs = pd.read_html(url)

df = pres_dfs[-10]   # code gets a different area than the book , because most likey the web page has changed.



In [ ]:
# the regular expression removes this notations if they exist "[any chars inside he brackets]""
df = (df
.iloc[1:-1]
.rename(columns={'Political party':'Party'})
.assign(Party = lambda df_: df_
    .Party
    .str.replace(r'\[.*\]', '')
    .astype('category'))
)

# export to csv
df.to_csv('siena2018-pres.csv')

# save a parquet instead

df.to_parquet('siena2018')



In [ ]:
df = pd.read_csv('siena2018-pres.csv', index_col=0)

df_p = pd.read_parquet('siena2018')



In [ ]:
pd.set_option('display.max_rows', 10)              # REMOVELINE 
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    'siena2018-pres.csv'
df = pd.read_csv(url, index_col=0)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))


    return (df
     .rename(columns={'Seq.':'Seq'})    # 1
     .rename(columns={k:v.replace(' ', '_') for k,v in
        {'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'}.items()})
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8')),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10), dpi=600)
g = sns.heatmap((tweak_siena_pres(df)     # doctest: +SKIP
   .set_index('President')
   .iloc[:,2:-1]
  ),annot=True, cmap='viridis', ax=ax)
g.set_xticklabels(g.get_xticklabels(), rotation=45, fontsize=8,
    ha='right')    # doctest: +SKIP
_ = plt.title('Presidential Ranking')
#fig.savefig('img/pandas2/20-pres.png', bbox_inches='tight')       # doctest: +SKIP

17.4 Exercises